# Recurrent Neural Networks

### Practical Session

<br/> Prof. Dr. Georgios K. Ouzounis
<br/> email: georgios.ouzounis@go.kauko.lt

## Contents

1. Data loading and pre-processing
2. Building the RNN
3. Train and deploy the RNN
4. Improving the RNN

## Challenge

<img src="https://www.sapientrade.com/images/2017/02/14/AI-Machine-Learning-Trading-Benefits.jpg" width="800" align="left"/>

Given a 10-year history of the Apple Stock prices predict the stock values for the period of the recent most month that are not included in the historical data.

## Data loading and pre-processing

I did not use the github, so i don't need to use github, to download the data.

### Import libraries

We need 3 main libraries:

- [Numpy](http://www.numpy.org): it is the fundamental package for scientific computing with Python. It contains among other things a powerful N-dimensional array object that can be used as an efficient multi-dimensional container of generic data. Arbitrary data-types can be defined.
- [matplotlib](https://matplotlib.org):  it is a Python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms.
- [pandas](https://pandas.pydata.org): is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series.

In [1]:
# library importation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Import the dataset

Dataset description: the open high, low and close values of the Google Stock from 2012 to 2016. [Relevant code here](https://github.com/pdway53/Predict_Google_Stock_Price_RNN)


In [2]:
# Dataset importation

# loading contents of the file for July 1st 2009 - July 31st 2019 
dataset_train = pd.read_csv('AAPL_10yrs.csv')

In [3]:
dataset_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-07-01,5.125000,5.166429,5.090000,5.101071,4.408037,414178800
1,2009-07-02,5.044643,5.101071,4.992500,5.000714,4.321311,370479200
2,2009-07-06,4.953571,4.963929,4.866071,4.950357,4.277796,498688400
3,2009-07-07,4.945714,4.988572,4.827857,4.835714,4.178730,461596800
4,2009-07-08,4.854286,4.930000,4.800714,4.900714,4.234900,575929200


In [4]:
# Creating a subtable of related openings
# The .values option causes this vector to form a numpy array
training_set = dataset_train.iloc[:, 1:2].values 

In [5]:
training_set

array([[ 5.125   ],
       [ 5.044643],
       [ 4.953571],
       ...,
       [51.869999],
       [52.115002],
       [52.189999]])

Numpy arrays do not support the view() and head() methods. [More on accessing the numpy data](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html)


<img src="https://drive.google.com/uc?id=1qb64Qywa-VEv2CG6M6tF0zSouWZC7X-S"/>

### Feature Scaling

Next we need to rescale our data to the range from 0 to 1. 

Feature scaling is essential as discussed if the Features lecture and needs to be applied to both the training and test sets.

It is computed using the ScikitLearn library [MinMaxScaler()](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) which transforms the selected feature by scaling it to a given range. If more than one, this estimator scales and translates each feature individually such that it is in the given range on the training set, i.e. between zero and one.

In [6]:
# Scaling the feature

# importing the MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

In [7]:
# Creating a scaler instance to rescale all data to the range of 0.0 to 1.0 
sc = MinMaxScaler(feature_range = (0, 1))

In [8]:
# Creating a certain training set of scaled values
training_set_scaled = sc.fit_transform(training_set)

In [9]:
training_set_scaled

array([[0.00512321],
       [0.00360247],
       [0.00187895],
       ...,
       [0.88976301],
       [0.89439965],
       [0.89581895]])

### Splitting the training set to dependent and independent variables

<img src="https://drive.google.com/uc?id=1bckuLGZCeLUzNA-xJCGOODzC-4n2U-If"/>

In [10]:
# Creating a data structure with 90 timesteps and 1 output

# The 90 stock prices in the last 3 months before today
X_train = [] 

In [11]:
training_set_scaled.shape

(2537, 1)

In [12]:
# Today's stock price
y_train = [] 

In [13]:
# Starting from day 90, then going back 90 days 
for i in range(90, 2537): 
    # Number 0 defines column ID, which is the only column in this case    
    # Putting the last 90 days values in one row of X_train
    X_train.append(training_set_scaled[i-90:i, 0]) 
    y_train.append(training_set_scaled[i, 0])


In [14]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [15]:
X_train

array([[0.00512321, 0.00360247, 0.00187895, ..., 0.03509875, 0.0370453 ,
        0.03817401],
       [0.00360247, 0.00187895, 0.00173026, ..., 0.0370453 , 0.03817401,
        0.03824837],
       [0.00187895, 0.00173026, 0.        , ..., 0.03817401, 0.03824837,
        0.04124254],
       ...,
       [0.79925521, 0.78922501, 0.8071563 , ..., 0.89439965, 0.89066196,
        0.89643403],
       [0.78922501, 0.8071563 , 0.83232624, ..., 0.89066196, 0.89643403,
        0.88976301],
       [0.8071563 , 0.83232624, 0.81420575, ..., 0.89643403, 0.88976301,
        0.89439965]])

### Reshaping the Matrix

We need to add a new matrix dimension to accommodate the indicator (predictor). 

NumPy matrices are tensors (3D) and essentially we need to specify that our matrix consists of **60 days** (dimension x) times **total days in data set** (dimension y) times **1 value per matrix cell (scalar)** (dimension z)

<img src="https://cdn.pixabay.com/photo/2015/03/22/17/34/cubic-684961_1280.jpg" width="500" align="left"/>

If we were to add the stock value of somebody else together with the the past 60 days of Google, we need to change the length of the 3 dimension to  2.  RNN training tables are 3D!!! Read: [Reshaping NumPy Array | Numpy Array Reshape Examples](https://backtobazics.com/python/python-reshaping-numpy-array-examples/)

In [16]:
# Reformating the data matrix, we retain the 2 original dimensions and adding a third one with the depth of 1
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Building the RNN

### RNN initialization

- Import the sequential model from the Keras API;
- Import the Dense layer template from the Keras API;
- Import the LSTM model from the Keras API
- Create an instance of the sequential model called regressor because we want to predict a continuous value

In [17]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [18]:
# Initialising the RNN as an arrangement of layers
regressor = Sequential()

### Add First Layer

We first add an object of the LSTM class! 

- The first argument is the number of units or LSTM memory cells. Include many neurons to address the high dimensionality of the problem; say 50 neurons! 
- Second arg: return sequences = true; stacked LSTM !
- Third arg: input 3D shape: observations vs time steps vs number of indicators

In [19]:
# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 50, return_sequences = True, input_shape =  (X_train.shape[1], 1)))

In [20]:
# The argument is the dropout rate to ignore in the layers (20%) 
# For example 100 units * 20% = 20 units will be dropped each time
regressor.add(Dropout(0.2))


### Add More Layers

We can add more LSTM layers but along with Dropout regularization to make sure we avoid overfitting! 

We don’t need to add the shape of the layer again because it is recognized automatically from the number of input units.

The last layer does not return a sequence but connected directly to a fully connected output layer.


In [21]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [22]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [23]:
# Adding a fourth LSTM layer and some Dropout regularisation
# I removed the return_sequences message, because we no longer return an arrangement, but a value instead
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

### Add Output Layer & Compile

The output has 1 dimension , i.e. one value to be predicted thus or output fully connected layer has dimensionality = 1.

- **Optimizer**: rmsprop is recommended in the Keras documentation. The Adam optimizer is also a powerful choice.
- **Loss function**: regression problems take the mean square error as most common

In [24]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [25]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

## Train and deploy the RNN

### Fit the RNN to the Training set

We now want to train our RNN using the data in our **Training Set X** and **predictors in y** (ground truth in this case). Parameters that can be specified are the:

- **Batch size**:  update the cell weights not on every stock price on every batch_size values; 
- **Number of epochs**: how many iterations to be used, i.e. number of forward and backward propagations for the update of the weights


In [ ]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
77/77 [==============================] - 10s 128ms/step - loss: 0.0177
Epoch 2/100
77/77 [==============================] - 9s 120ms/step - loss: 0.0033
Epoch 3/100
77/77 [==============================] - 9s 119ms/step - loss: 0.0031
Epoch 4/100
77/77 [==============================] - 9s 121ms/step - loss: 0.0029
Epoch 5/100
77/77 [==============================] - 10s 125ms/step - loss: 0.0027
Epoch 6/100
77/77 [==============================] - 10s 125ms/step - loss: 0.0026
Epoch 7/100
77/77 [==============================] - 9s 122ms/step - loss: 0.0024
Epoch 8/100
77/77 [==============================] - 10s 124ms/step - loss: 0.0027
Epoch 9/100
77/77 [==============================] - 9s 121ms/step - loss: 0.0021
Epoch 10/100
77/77 [==============================] - 9s 122ms/step - loss: 0.0020
Epoch 11/100
77/77 [==============================] - 10s 127ms/step - loss: 0.0022
Epoch 12/100
77/77 [==============================] - 10s 129ms/step - loss: 0.0020
Epoch 1

77/77 [==============================] - 9s 120ms/step - loss: 6.7411e-04
Epoch 97/100
77/77 [==============================] - 9s 116ms/step - loss: 6.8634e-04
Epoch 98/100
77/77 [==============================] - 10s 128ms/step - loss: 6.4713e-04
Epoch 99/100
42/77 [===============>..............] - ETA: 4s - loss: 7.0884e-04

### Computing Predictions

Create a data-frame by importing the Apple Stock Price Test set for August 2019 using pandas and make it a numpy array.

There are 22 financial days in one month, weekends are excluded!


In [ ]:
# Getting the real stock price for August 1st 2019 - August 31st 2019

dataset_test = pd.read_csv('AAPL_test.csv')
dataset_test

In [ ]:
real_stock_price = dataset_test.iloc[:, 1:2].values
real_stock_price.size

In [ ]:
real_stock_price

To predict the stock price value for each day in August 2019, we need the values in the last 90 days.

To obtain this **history** we need to combine both the training and test sets in one.

If we were to use the training_set and test_set we would need to use the scaler  but that would change the actual test values.  Thus concatenate the original data frames!


In [ ]:
# Getting the predicted stock price of the future

# Axis = 0 means concatenate the lines (for example: vertical axis)
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0) 

In [ ]:
dataset_total.size


In [ ]:
# The contrast in the length of the first two gives us the first day in July 2019, and we need to go back 90 days to get the necessary range
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 90:].values

In [ ]:
inputs.size

In [ ]:
# I did not use iloc from panda so lets rearrange the numpy array for compatibility, for example: all the values from input lines to be stacked in one column. The -1 means that the numpy has no knowledge of how the values were stored in lines. The 1 means I want to them in one column

inputs = inputs.reshape(-1,1) 

# Applying the feature scaler
inputs = sc.transform(inputs)

1. For each price in Jan. 2017 we need the **immediate 90 values** before it. 
2. We have 21 prices in August;
3. We need a numpy 3D array of 90 prices (columns) times 21 days (rows) times 1 dependent variable 
4. We don’t need y_test. That is what we are trying to compute!


In [ ]:
# Getting the predicted stock price of the future
X_test = []

In [ ]:
# The first 90 from inputs are from training set, starting from 90 and getting the extra 22
for i in range(90, 112): 
    X_test.append(inputs[i-90:i, 0])


In [ ]:
X_test = np.array(X_test)

In [ ]:
# Creating a 3D structure
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted_stock_price = regressor.predict(X_test)

In [ ]:
# I need to inverse the scaling to get useful predicted stock price
predicted_stock_price = sc.inverse_transform(predicted_stock_price) 
predicted_stock_price.size

### Result Visualization


In [ ]:
# Visualisation of the results by graph

plt.plot(real_stock_price, color = 'red', label = 'Real Apple Stock Price')

plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Apple Stock Price')

plt.title('Apple Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Apple Stock Price')
plt.legend()

plt.show()

The blue line shows the trend of the stock for the month of August 2019. 

Some observations:
- The prediction lags behind the actual price curve because the model cannot react to fast non-linear changes. Spikes are examples of fast non-linear changes
- Model reacts pretty well to smooth changes

### Compute the RMSE

If we need to compute the RMSE for our Stock Price Prediction problem, we use the real stock price and predicted stock price as shown.

Then consider dividing this RMSE by the range of the Apple Stock Price values of August 2019 to get a relative error, as opposed to an absolute error. 


In [ ]:
# Importing the libraries
import math
from sklearn.metrics import mean_squared_error

In [ ]:
rmse = math.sqrt( mean_squared_error( real_stock_price[0:22,:], predicted_stock_price))
rmse

## Improving the RNN

The new data need to be placed in the same order/format  as in the case of the training/test sets.

1. Getting more training data: we trained our model on the past 10 years of the  Apple Stock Price but it would be even better to train it on the past 10 years.

2. Increasing the number of time steps: the model remembered the stock price from the 60 previous financial days to predict the stock price of the next day. That’s because we chose a number of 90 time steps. You could try to increase the number of time steps, by choosing for example 120 time steps (6 months).

3. Adding some other indicators: if you have the financial instinct that the stock price of some other companies might be correlated to the one of Apple, you could add this other stock price as a new indicator in the training data.

4. Adding more LSTM layers: we built a RNN with four LSTM layers but you could try with even more.

5. Adding more neurons in the LSTM layers: we highlighted the fact that we needed a high number of neurons in the LSTM layers to respond better to the complexity of the problem and we chose to include 50 neurons in each of our 4 LSTM layers. You could try an architecture with even more neurons in each of the 4 (or more) LSTM layers.

### Fine tuning the RNN

Parameter Tuning on the RNN model: we are dealing with a Regression problem because we predict a continuous outcome.

**Tip**: replace: scoring = 'accuracy' by scoring = 'neg_mean_squared_error' in the GridSearchCV class parameters as we did in the ANN case.